# Google Smart Composer

<img src = "https://drive.google.com/open?id=1xw4Jh-0rRhA1e5X1lH8bLjwmyeSCcE62">


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.translate.bleu_score import sentence_bleu as score
from tqdm import tqdm
import sys
import re
import numpy as np
import os
import io
import time


import warnings
warnings.filterwarnings('ignore')

TensorFlow 2.x selected.


## Data: 
Data is taken from personal emails and constructed according to problem specific.

### Structure of data:
From all the emails 'to', 'subject', 'previous email'(if any) and 'content' parts are taken. As we are going to predict next comming word in the content  part, content part is breaked into following way. We are restriction our self to predict only atmost 5 next comming words so each email content will be breaked to many sentances.
<br><b>Ex:</b> Content: This is introduction to my project<br>
 <table>
    <tr>
      <th>Sentance</th>
      <th>Output</th>
    </tr>
    <tr>
      <td> This </td>
      <td> is</td>
    </tr>
    <tr>
      <td> This </td>
      <td> is introduction </td>
    </tr>
    <tr>
      <td> This </td>
      <td> is introduction to</td>
    </tr>
    <tr>
      <td> This </td>
      <td> is introduction to my</td>
    </tr>
    <tr>
      <td> This </td>
      <td> is introduction to my project</td>
    </tr>
    <tr>
      <td> This is </td>
      <td> introduction</td>
    </tr>
    <tr>
      <td> This is</td>
      <td> introduction to </td>
    </tr>
    <tr>
      <td> This is </td>
      <td> introduction to my</td>
    </tr>
    <tr>
      <td> This is</td>
      <td> introduction to my</td>
    </tr>
    <tr>
      <td> This is</td>
      <td> introduction to my project</td>
    </tr>
    <tr>
      <td> This is introduction</td>
      <td> to</td>
    </tr>
    <tr>
      <td> This is introduction</td>
      <td>  to my </td>
    </tr>
    <tr>
      <td> This is introduction </td>
      <td> to my project</td>
    </tr>
    
   
 </table>

Now to the sentance part 'to','subject', 'previous email' parts are joined with their corresponding separaters.<br>
<b>Ex:</b> < to > email@ email.com< sub > introduction < prv > nan < cont > hello email this is only an intro 





In [0]:
from google.colab import drive
drive.mount('gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrive


In [0]:
from google.colab import files
files.download('gdrive/My Drive/google/final_data_my_emails.csv')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 57514, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
device_name = tf.test.gpu_device_name()
print(device_name)

## Loading Data

In [0]:
data = pd.read_csv('gdrive/My Drive/google/final_data_my_emails.csv')
print('Number of rows in data',data.shape[0])
print('Number of columns in data',data.shape[1])
data.head()

Number of rows in data 181255
Number of columns in data 2


,x,y
0,<to> yernagulahemanth <sub> how are you commi...,"yernagulahemanth,"
1,<to> yernagulahemanth <sub> how are you commi...,"yernagulahemanth, i"
2,<to> yernagulahemanth <sub> how are you commi...,"yernagulahemanth, i will"
3,<to> yernagulahemanth <sub> how are you commi...,"yernagulahemanth, i will be"
4,<to> yernagulahemanth <sub> how are you commi...,"yernagulahemanth, i will be coming"


In [0]:
data.y = data.y.apply(lambda x:str(x)) # y part was having now int values so they are converted to string values

# Along with defalt tags x and y are added with <start> and <end> tags at starting and ending 
data.x = data.x.apply(lambda x:'<start> ' + str(x) + ' <end>')
data.y = data.y.apply(lambda x:'<start> ' + str(x) + ' <end>')
data.head()

,x,y
0,<start> <start> <to> yernagulahemanth <sub> h...,"<start> <start> yernagulahemanth, <end> <end>"
1,<start> <start> <to> yernagulahemanth <sub> h...,"<start> <start> yernagulahemanth, i <end> <end>"
2,<start> <start> <to> yernagulahemanth <sub> h...,"<start> <start> yernagulahemanth, i will <end..."
3,<start> <start> <to> yernagulahemanth <sub> h...,"<start> <start> yernagulahemanth, i will be <..."
4,<start> <start> <to> yernagulahemanth <sub> h...,"<start> <start> yernagulahemanth, i will be c..."


In [0]:
# Tokenizing x and y
# Tokenizing: Collecting all tokens(words) for x and y and storing them  with index number

X_  = list(data.x.values)
Y_ = list(data.y.values)

x_tokenizer   = tf.keras.preprocessing.text.Tokenizer(filters='')
x_tokenizer.fit_on_texts(X_)
x_tensor         = x_tokenizer.texts_to_sequences(X_)

# padding
x_tensor         = tf.keras.preprocessing.sequence.pad_sequences(x_tensor,padding='post')


y_tokenizer  = tf.keras.preprocessing.text.Tokenizer(filters='')
y_tokenizer.fit_on_texts(Y_)
y_tensor        = y_tokenizer.texts_to_sequences(Y_)
y_tensor        = tf.keras.preprocessing.sequence.pad_sequences(y_tensor,padding='post')


In [0]:
print('Total number of words in x:',len(x_tokenizer.index_word))
print('Total number of words in y:',len(y_tokenizer.index_word))

Total number of words in x: 6082
Total number of words in y: 6006


In [0]:
# Making train and validation data with 80-20 ratio 
x_train, x_val, y_train, y_val = train_test_split(x_tensor, y_tensor, test_size=0.2)

# Show length
y_token_max_len, x_token_max_len = max([len(i) for i in y_tensor]), max([len(i) for i in x_tensor])

print('Number of data points in xtrain:',len(x_train))
print('Number of data points in ytrain:',len(y_train))
print('Number of data points in xval  :',len(x_val))
print('Number of data points in yval  :',len(y_val))

Number of data points in xtrain: 145004
Number of data points in ytrain: 145004
Number of data points in xval  : 36251
Number of data points in yval  : 36251


In [0]:
y_token_max_len,x_token_max_len

(9, 803)

## Feeding this data into tensorflow data api
we can make operation like shuffleing, getting batch wise so as to make future simple

`Ref: https://www.tensorflow.org/guide/data'

In [0]:
Batch_Size = 100
steps_per_epoch = len(x_train)//Batch_Size
Emd_dim = 200
units = 500
x_vocab_size = len(x_tokenizer.word_index)+1
y_vocab_size = len(y_tokenizer.word_index)+1

# Shuffling the data set 
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train))
# Getting batch wise and droping last batch if it is not having less number of points(diss similar batch)
dataset = dataset.batch(Batch_Size, drop_remainder=True)

In [0]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([100, 803]), TensorShape([100, 9]))

## Encoder Decoder With ATTENTION
---------put that big image here ------

In [0]:
class Encoder(tf.keras.Model):
  '''
    vocab_size   - Vocabulary size
    Emd_dim      - Number of emding dimensions
    batch_size   - Batch Size
    
  '''
  def __init__(self, vocab_size, Emd_dim, enc_units, batch_size,eps = 0.001,drp_rt = 0.2):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, Emd_dim)
    # self.embedding = MyLayer(vocab_size,Emd_dim,enc_units)
    self.encoder = tf.compat.v1.keras.layers.CuDNNLSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.batch_norm = tf.keras.layers.LayerNormalization(epsilon=eps)
    self.dropout = tf.keras.layers.Dropout(rate=drp_rt)
  
  def call(self, input_, hidden,train):

    '''
      Given input and hidden states for encoder returns the updated hidden states and output
    '''
    # with tf.device('/gpu:0'):
    input_ = self.embedding(input_)
    # print(input_.shape)
    # print(x.shape,hidden[0].shape,hidden[1].shape)
    output, state_h,state_c = self.encoder(input_, initial_state = hidden)
    # output = self.dropout(output,training = train)
    # output = self.batch_norm(output)
    # state = tf.concat([state_h,state_c],axis=1)
    state = [state_h,state_c]
    return output, state

 
  def initialize_hidden_state(self):
      '''
       Returns initial states for encoder
      '''
      initial_states = [tf.zeros((self.batch_size, self.enc_units)),tf.zeros((self.batch_size, self.enc_units))]
      return initial_states



In [0]:
encoder = Encoder(x_vocab_size, Emd_dim, units, 100)

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden,False)


print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
# print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))


NotFoundError: ignored

In [0]:
class Attention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(Attention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
    self.units = units

  def call(self, hid, enc_out):

    # hidden shape == (Batch_Size, hidden size)
    # hidden_with_time_axis shape == (Batch_Size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    # hidden_with_time_axis = tf.expand_dims(hid, 1)
    # Since Lstm layers will give us two states i.e cell state and hidden state 
    # we are passing hidden state through dense layer and cell state through dence state
    # and adding both outputs of dence  layer
    hidden_with_time_axis  = tf.expand_dims(self.W2(hid[0])+self.W2(hid[1]),1)
    # print(hidden_with_time_axis.shape)
    # Now this sum of  outputs from dense layers is added with outputs of  encoder output and 
    # along with tanh and a single unit dense layer
    # score shape == (Batch_Size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(enc_out) + hidden_with_time_axis))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * enc_out
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
attention_layer = Attention(10)
sample_context, sample_atweights = attention_layer(sample_hidden, sample_output)
print('Attention context  shape:',sample_context.shape)
print('Attention weights shape :',sample_atweights.shape)

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, Emd_dim, dec_units, batch_size,nwtpa = 1000,eps=0.001,drp_rt=0.1):
    ''' 
    All the variables are same as previous encoder calss except nwtpa
    nwtpa is number of units you want to maintain at attention mechanism
    '''

    super(Decoder, self).__init__()
    self.batch_size = batch_size
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, Emd_dim)
    # self.embedding = MyLayer(vocab_size,  Emd_dim,dec_units)
    self.decoder =  tf.compat.v1.keras.layers.CuDNNLSTM(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.batch_norm = tf.keras.layers.LayerNormalization(epsilon=eps)
    self.dropout    = tf.keras.layers.Dropout(drp_rt)
    # self.time_dist  = (tf.keras.layers.Dense(2),input_shape=(100,500))
    self.fc =tf.keras.layers.Dense(vocab_size,activation='softmax')
    self.nwtpa = nwtpa
    self.time_dist1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(50))
    self.time_dist2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(15))
    # used for attention
    self.attention = Attention(self.nwtpa)

  def call(self, input_, hidden, enc_output):
    with tf.device('/gpu:0'):
        # enc_output shape == (batch_size, max_length, hidden_size)
        # passing  hidden states and encoder output to the attention layer
        context_vector, attention_weights = self.attention(hidden, enc_output)
        # context_vector = self.batch_norm(context_vector)
        # context_vector = self.dropout(context_vector)
        # applying embdding to the inputs of the dense layer
        # input_ shape after passing through embedding == (batch_size, 1, Emd_dim)
        input_ = self.embedding(input_)

        # concating previous input and context vector
        # input_ shape after concatenation == (batch_size, 1, Emd_dim + hidden_size)
        input_ = tf.concat([tf.expand_dims(context_vector, 1), input_], axis=-1)

        # passing the concatenated vector to the Lstm
        output, state_h,state_c = self.decoder(input_)
        state = [state_h,state_c]

        # output shape == (batch_size * 1, hidden_size)
        output = self.time_dist1(output)
        output = self.time_dist2(output)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        # output = self.time_dist(output)
        output = self.fc(output)

        return output, state, attention_weights

In [0]:
decoder = Decoder(y_vocab_size, Emd_dim, units, Batch_Size)
sample_deout, sample_destate,sample_atweights_ = decoder(tf.random.uniform((Batch_Size, 1)),sample_hidden,sample_output)
print('shape of decoder output:',sample_deout.shape)
# print('shape of decoder state :',sample_destate.shape)


## Define the optimizer and the loss function

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(actual, pred):
  with tf.device('/gpu:0'):
      mask = tf.math.logical_not(tf.math.equal(actual, 0))
      loss_ = loss_object(actual, pred)

      mask = tf.cast(mask, dtype=loss_.dtype)
      loss_ *= mask

      return tf.reduce_mean(loss_)

## Creating Checkpoints

In [0]:
import time
# print(str(time.localtime().tm_hour)+'_'+str(time.localtime().tm_min)+'_'+str(time.localtime().tm_sec))
checkpoint_dir = 'gdrive/My Drive/google/checkpoints/check'

checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

## Training


In [0]:
def test(sentence):
  with tf.device('/gpu:0'):
      
      # Tokenizing inputs
      # inputs = [x_tokenizer.word_index[i] for i in sentence.split()]
      # Padding inputs
      inputs = sentence
      inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                            maxlen=x_token_max_len,
                                                            padding='post')
      inputs = tf.convert_to_tensor(inputs)

      result = ''

      # Making initial hidden states for encoder
      hidden = [tf.zeros((1, units)),tf.zeros((1, units))]
      
      # inputs and hidden states are passed to encoder
      enc_out, enc_hidden = encoder(inputs, hidden,train = False)
      

      dec_hidden = enc_hidden

      # Making inputs for decoder as <start>
      dec_input = tf.expand_dims([y_tokenizer.word_index['<start>']], 0)

      
      for t in range(y_token_max_len):
              # inputing decoder input(<start> during initial sate), encoder hidden state and encoder output
              predictions, dec_hidden,_ = decoder(dec_input, dec_hidden,enc_out)
      
              predicted_id = tf.argmax(predictions[0]).numpy()
              

              result += y_tokenizer.index_word[predicted_id] + ' '

              # if the predicted word is <end> returning the words predicted till now
              if y_tokenizer.index_word[predicted_id] == '<end>':
                return result

              # current output from decoder is fed back to decoder into next timestep
              dec_input = tf.expand_dims([predicted_id], 0)
      
      return result



def predict_next_word(sentence):
      result = test(sentence)
      return result


def bleu_score(actual,pred):
#     print('Actual:',actual)
#     print('Predic:',pred)
    return score([actual],pred)


def xgenerate_sent(data,id_):

        test_with = []
        try:
          for i in data[id_]:
            test_with.append(x_tokenizer.index_word[i])
        except:

          return ' '.join(test_with)

        return ' '.join(test_with)

def ygenerate_sent(data,id_):

        test_with = []
        
        try:
          for i in data[id_]:
            test_with.append(y_tokenizer.index_word[i])
        except:
          
          return ' '.join(test_with)
        
        return ' '.join(test_with)        


def log_perplexity(prob):
      
      return np.dot(np.log(prob),prob)




def score_calc (type_,x=0,y=0,nm='0'):

      total_score = []
      if type_   == 'log_perplex':
          
          for i in tqdm(range(len(x)),position=0):
              
              total_score.append(log_perplexity(test(x[i])[1]))
              
          return -np.mean(total_score)
          

      elif type_ == 'bleu':
          
          for i in tqdm(range(len(x)),position=0):
                  
                  total_score.append(bleu_score(y[i],test(x[i])[0]))
          return np.mean(total_score)

In [0]:
x_train_ = []
y_train_ = []
for i in tqdm(range(x_train.shape[0]),position=0):
    x_train_.append(xgenerate_sent(x_train,i))
    y_train_.append(ygenerate_sent(y_train,i))


x_val_ = []
y_val_ = []
for i in tqdm(range(x_val.shape[0]),position=0):
    x_val_.append(xgenerate_sent(x_val,i))
    y_val_.append(ygenerate_sent(y_val,i))    



In [0]:
# inputs = x_train_[0]
# inputs = [x_tokenizer.word_index[i] for i in inputs.split()]
# inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
#                                                             maxlen=x_token_max_len,
#                                                             padding='post')
# inputs = tf.convert_to_tensor(inputs)

# inputs

In [0]:
def train(EPOCHS = 10,verbose = 0,optimizer= tf.keras.optimizers.Adam()):
    with tf.device('/gpu:0'):
        initial_score = 10
        score_500     = 0 
        logdir = "logs/train_data/"
        model_name = str(optimizer).split('object at')[0].split('.')[-1].strip()
        train_writer = tf.summary.create_file_writer(logdir+str('model_{}'.format(model_name)))
        with train_writer.as_default():
          for epoch in range(EPOCHS):
                print('\nTraining Epoch: {}'.format(epoch+1))
                epoch_score = []
                start = time.time()

                enc_hidden = encoder.initialize_hidden_state()
                total_loss = 0

                for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
                        # batch_loss = train_step(inp, targ, enc_hidden)
                        
                        
                        loss = 0
                      
                        with tf.GradientTape() as tape:
                              enc_output, enc_hidden = encoder(inp, enc_hidden,train = False)
                              # print(enc_output[0][0][0])
                              dec_hidden = enc_hidden
                              
                              dec_input = tf.expand_dims([y_tokenizer.word_index['<start>']] * Batch_Size, 1)
                              for t in range(1, targ.shape[1]):
                                  # passing enc_output to the decoder
                                  predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

                                  loss += loss_function(targ[:, t], predictions)

                                  # using teacher forcing
                                  dec_input = tf.expand_dims(targ[:, t], 1)

                              
                        
                        batch_loss = (loss / int(targ.shape[1]))

                        variables = encoder.trainable_variables + decoder.trainable_variables

                        gradients = tape.gradient(loss, variables)

                        optimizer.apply_gradients(zip(gradients, variables))
                        total_loss += batch_loss
                        
                        if verbose:
                            
                            if batch % verbose == 0:
                                  
                                  print('Batch {} Loss {:.4f}'.format(batch,batch_loss.numpy()))

                tf.summary.scalar('loss',total_loss/steps_per_epoch,step=epoch)
                checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt".format(str(time.localtime().tm_hour)+'_'+str(time.localtime().tm_min)+'_'+str(time.localtime().tm_sec)))
                checkpoint.save(file_prefix = checkpoint_prefix)

                
                if (epoch + 1 ) % 5 == 0:
                    print('\n')
                    print('Calculating score for train and test data it may take some time...')
                    train_score = score_calc(type_ = 'bleu',x = x_train,y = y_train_)
                    val_score   = score_calc(type_ = 'bleu',x = x_val,y = y_val_)
                    tf.summary.scalar('Train_score',train_score,step=epoch)
                    tf.summary.scalar('Test_score',val_score,step=epoch)
                    print('----'*15)
                    print('Epoch {} Loss {:.4f} Train Score {} Test Score {} Time {} sec'.format(epoch + 1, total_loss / steps_per_epoch,train_score,val_score,time.time() - start))
                    print('----'*15)
                
                else:
                  print('----'*15)
                  print('Epoch {} Loss {:.4f} Time {} sec'.format(epoch + 1, total_loss / steps_per_epoch, time.time() - start))
                  print('----'*15)

                writer_flush = train_writer.flush()
                

In [0]:
# %load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir logs/train_data/


In [0]:
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
train(EPOCHS=10,verbose = 100)

In [0]:
# function ConnectButton(){
#     console.log("Connect pushed"); 
#     document.querySelector("#connect").click() 
# }
# setInterval(ConnectButton,60000);

## Restore the latest checkpoint and test

In [0]:
# # restoring the latest checkpoint in checkpoint_dir
# checkpoint_dir = 'check_points'
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
# import shutil
# # Moving checkpoints to drive
# for i in os.listdir('checkpoints'):
#   shutil.move('checkpoints/'+i,'gdrive/My Drive/google/check_points/with_to_missplaced/')

In [0]:
# # Correct     : 5,6,12,15,16,18,20,27,35,37,40,41
# # Little Wrong: 4,8,10,14,29
# # Interesting : 32,38
# id_ = 1
# print(x_val_[id_])
# # print(x_val_[id_][:-9])
# print('Actual y:',y_val_[id_])
# result, pred,pred_prob = predict_next_word(x_val_[id_])
# print('Pred   y:',result)


# Log = np.log(pred_prob)
# print(np.dot(np.nan_to_num(Log),pred_prob))
# print("Log",Log)
# print("Prob",pred_prob)
# final_perp = []
# for i in [5,6,12,15,16,18,20,27,35,37,40,41]:
#   result,pred,pred_prob = predict_next_word(x_val_[i])
#   Log = np.log(pred_prob)
#   print('Actual y :',y_val_[i])
#   print('Pred   y :',result)
#   print('Pred_prob:',[j.numpy() for j in pred_prob])
#   print('Log prob :',Log)
#   print(np.dot(Log,pred_prob))
#   final_perp.append(np.dot(Log,pred_prob))
#   print('='*10)

# -sum(final_perp)/len(final_perp)


## We shall check the  model with some unseen data

<strong> Most of the predictions are correct, every time model id predicting 3 or more words</strong> <br>NOTE: In actual section i have placed all next comming words(to make understad for readers) model only predicts minimum one word and maximum five words

<strong>Pytorch takes less time when compared to Tensorflow for  Google NMT models:</strong>https://medium.com/syncedreview/tensorflow-pytorch-or-mxnet-a-comprehensive-evaluation-on-nlp-cv-tasks-with-titan-rtx-cdf816fc3935